# Goal
This notebook introduces 3D brain MRI handling with MONAI:
- Load T1/T2 NIfTI volumes
- Minimal 3D preprocessing (orientation, spacing, intensity)
- Visualize axial slices
- Optionally export 2D PNG slices for the 2D GAN notebooks (Pix2Pix/CycleGAN)


In [18]:
# Colab setup: clone repo, install deps, and create sample data
import os, sys, subprocess
from pathlib import Path

IN_COLAB = False
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    pass

repo_url = "https://github.com/julietam/GenerativeAI_Medical_Images.git"
workdir = Path("/content/GenerativeAI_Medical_Images") if IN_COLAB else Path.cwd()

if IN_COLAB:
    if not workdir.exists():
        subprocess.run(["git", "clone", repo_url, str(workdir)], check=True)
    else:
        subprocess.run(["git", "-C", str(workdir), "pull", "--ff-only"], check=False)
    os.chdir(workdir)

subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"], check=False)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "torchmetrics", "monai", "nibabel", "scikit-image", "Pillow"], check=False)
subprocess.run([sys.executable, "scripts/make_sample_data.py"], check=False)

print("Setup done. CWD =", Path.cwd())
print("Sample roots: data/sample_mri_pairs, data/sample_unpaired_mri, data/sample_unpaired_ct")


Setup done. CWD = /content/GenerativeAI_Medical_Images
Sample roots: data/sample_mri_pairs, data/sample_unpaired_mri, data/sample_unpaired_ct


# 01_introduction_mri.ipynb — MONAI transforms on IXI

Pequeña introducción a carga y preprocesamiento de MRI 3D con MONAI, y extracción de slices 2D para notebooks de este repo.


In [19]:
import os, torch
!pip install monai
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd, ScaleIntensityRanged, CropForegroundd, Resized
from monai.data import Dataset, DataLoader
from monai.utils import first
import matplotlib.pyplot as plt
from pathlib import Path


## 1) Paths y lista de archivos (IXI T1/T2)
Ajusta `ixi_root` a tu carpeta con NIfTI (T1/T2).


In [20]:
# Pares de T1/T2 en PNG (preferido por los notebooks 03/entrenamiento)
paired_root = 'data/paired_mri'
from pathlib import Path
import os, subprocess, sys
t1_dir = Path(paired_root)/'T1'
t2_dir = Path(paired_root)/'T2'
sample_root = Path('data/sample_mri_pairs')
# Fallback: si no hay PNGs en paired_mri, intenta generar muestras y usar sample_mri_pairs
if not any(t1_dir.glob('*.png')) or not any(t2_dir.glob('*.png')):
    try:
        nb_cwd = Path.cwd()
        candidates = [nb_cwd/'scripts'/'make_sample_data.py', nb_cwd.parent/'scripts'/'make_sample_data.py']
        for sp in candidates:
            if sp.exists():
                subprocess.run([sys.executable, str(sp)], check=False)
                break
    except Exception as e:
        print('Warning: could not generate sample data:', e)
    if any((sample_root/'T1').glob('*.png')) and any((sample_root/'T2').glob('*.png')):
        paired_root = str(sample_root)
        t1_dir = sample_root/'T1'
        t2_dir = sample_root/'T2'
print('paired PNGs:', len(list(t1_dir.glob('*.png'))), 'T1 and', len(list(t2_dir.glob('*.png'))), 'T2 in', paired_root)

# (Opcional) IXI NIfTI root para la sección MONAI 3D de abajo
ixi_root = 'data/IXI'  # Coloca aquí los NIfTI (T1/T2)
img_size = 256
# recolectar unos pocos sujetos
from glob import glob
from pathlib import Path
Path(ixi_root).mkdir(parents=True, exist_ok=True)
files = sorted(glob(os.path.join(ixi_root, '**', '*T1*.nii*'), recursive=True))[:5]
files2 = sorted(glob(os.path.join(ixi_root, '**', '*T2*.nii*'), recursive=True))[:5]
print('found', len(files), 'T1 and', len(files2), 'T2 files')


paired PNGs: 0 T1 and 0 T2 in data/paired_mri
found 0 T1 and 0 T2 files


## 1.a) Descarga rápida (IXI)

- Sitio oficial: https://brain-development.org/ixi-dataset/
- Copia algunos archivos NIfTI T1/T2 a `data/IXI/` y re-ejecuta la celda anterior.
- Alternativa: usa tutorials de MONAI para descargar y luego copia unos NIfTI aquí.


## 2) Definir transforms MONAI (3D)
- Reorientar a RAS
- Espaciado isotrópico aproximado
- Normalizar intensidades
- Crop foreground
- Resize a tamaño uniforme


In [ ]:
transforms_t1 = Compose([
    LoadImaged(keys=['img']),
    EnsureChannelFirstd(keys=['img']),
    Orientationd(keys=['img'], axcodes='RAS'),
    Spacingd(keys=['img'], pixdim=(1.0,1.0,1.5), mode='bilinear'),
    ScaleIntensityRanged(keys=['img'], a_min=0, a_max=2000, b_min=0.0, b_max=1.0, clip=True),
    CropForegroundd(keys=['img'], source_key='img'),
    Resized(keys=['img'], spatial_size=(img_size, img_size, -1)),
])


## 3) Cargar un volumen y visualizar slices


In [ ]:
data = [{'img': f} for f in files]
ds = Dataset(data=data, transform=transforms_t1)
dl = DataLoader(ds, batch_size=1, shuffle=False)
item = first(dl)
vol = item['img'][0]  # (C, H, W, D)
print('volume shape:', vol.shape)
# mostrar 6 cortes
import numpy as np
C,H,W,D = vol.shape
idxs = np.linspace(D*0.2, D*0.8, 6, dtype=int)
fig, axes = plt.subplots(2,3, figsize=(9,6))
for ax, z in zip(axes.ravel(), idxs):
    ax.imshow(vol[0,:,:,z], cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'z={z}')
    ax.axis('off')
plt.tight_layout()
plt.show()


## 4) (Opcional) Exportar PNGs 2D para notebooks
Esto replica `src/data/datasets.py`, pero aquí mostramos cómo hacerlo con MONAI.


In [ ]:
out_dir = Path('data/monai_exports')
out_dir.mkdir(parents=True, exist_ok=True)
for i, batch in enumerate(dl):
    vol = batch['img'][0]  # (1,H,W,D) en [0,1]
    D = vol.shape[-1]
    zs = [D//2]  # centro
    for z in zs:
        sl = vol[0,:,:,z].numpy()
        from PIL import Image
        Image.fromarray((sl*255).astype('uint8')).save(out_dir / f'subj{i}_z{z:03d}.png')
print('Exported to', out_dir)
